In [ ]:
include("_.jl")

In [ ]:
me_ge_ = OrderedDict{String,Vector{String}}()

;

In [ ]:
en = Gene.read_ensembl(; or = "mouse")

pr_ = unique(skipmissing(en[en[!, "Gene type"].=="protein_coding", "Gene name"]))

pr_ = sort(string.(pr_))

## Ensembl

In [ ]:
mo_hu = Gene.map_mouse()

me_ge_["Ensembl"] = [pr for pr in pr_ if !haskey(mo_hu, pr)]

## Mouse Genome Informatics

In [ ]:
mg = TableAccess.read(joinpath(PAI, "hom_mousehumansequence.rpt.txt.gz"))

se_ = mg[!, "Common Organism Name"] .== "human"

mg[se_, "Symbol"] .= Gene.rename(string.(mg[se_, "Symbol"]))[1]

mo_hu = Dict{String,String}()

for da in groupby(mg, "DB Class Key")

    sy_ = da[!, "Symbol"]

    se_ = da[!, "Common Organism Name"] .== "mouse, laboratory"

    for mo in sy_[se_], hu in sy_[.!se_]

        mo_hu[mo] = hu

    end

end

println(length(mo_hu))

me_ge_["MGI"] = [pr for pr in pr_ if !haskey(mo_hu, pr)]

## Write

In [ ]:
write_gene_by_method(PAO, "mouse", me_ge_)